In [156]:
from yahoofinancials import YahooFinancials
import numpy as np

In [268]:
class coolFinancials(YahooFinancials):
    ticker = ''
    bshData = []
    def __init__(self, ticker):
        if isinstance(ticker, str):
            self.ticker = ticker.upper()
        else:
            self.ticker = [t.upper() for t in ticker]
        self._cache = {}
        
    def get_balanceSheetHistory(self):
        self.bshData = self.get_financial_stmts('annual', 'balance')['balanceSheetHistory']
        return self.bshData
    
    def get_BV(self):
        bv = []
        if not(self.bshData):
            self.get_balanceSheetHistory()
        for i in range(np.size(self.bshData[self.ticker])):
            date_key = list(self.bshData[self.ticker][i].keys())[0]
            bv.append(self.bshData[self.ticker][i][date_key]['totalStockholderEquity'])
        return bv   
        
    def get_BV_GR(self, bv):
        return [np.size(bv)-1, (bv[0] - bv[-1])/(bv[-1]*(np.size(bv)-1))]
    
    def get_purchase_price(self, cEPS, growth, years, rRate, PE, safty):
        fEPS = cEPS*(1+growth)**years
        fPrice = fEPS*PE;
        stickerPrice = fPrice/(1+rRate)**years
        return stickerPrice, stickerPrice*safty
    
    def get_PE(self):
        return (self._stock_summary_data('trailingPE')+self._stock_summary_data('forwardPE'))/2
            
            
        

In [270]:
x = coolFinancials('TSLA')

In [271]:
bv = x.get_BV()

In [272]:
bv.insert(0, x.get_book_value())
print(bv)

[16031000000, 6618000000, 4923000000, 4237242000, 4752911000]


In [273]:
bvgr = x.get_BV_GR(bv)

In [279]:
growth = bvgr[1]
cEPS = x.get_earnings_per_share()
years = 8;
rRate = 0.2;
safty = 0.5
PE = x.get_PE()

In [275]:
x.get_pe_ratio()
x.get_PE()

634.01323

In [280]:
print(x.get_purchase_price(cEPS, growth, years, rRate, PE, safty))

(3201.5198354392187, 1600.7599177196093)
